In [1]:
import pyam
import pandas as pd

import tradeoff.ecpc_calculator as tecpc

import matplotlib.pyplot as plt
from pathlib import Path

<IPython.core.display.Javascript object>

Step 1: Read in the data

In [2]:
df = pyam.IamDataFrame(
    Path(
        '../data/101_data.xlsx'
    )    
)

pyam - INFO: Running in a notebook, setting up a basic logging at level INFO
pyam.core - INFO: Reading file ../data/101_data.xlsx
pyam.core - INFO: Reading meta indicators


Step 2: Filter out the necessary variables.

In [3]:
df_for_equity = df.filter(
    region='R5*',
    variable=[
        'AR6 Reanalysis|OSCARv3.2|Emissions|CO2|Direct Only',
        'Population'
    ]
)

In [4]:
df_for_equity.interpolate(
    time=range(2005, 2101),
    inplace=True
)

Step 3: Aggregate a "World" region.

In [5]:
df_for_equity.aggregate_region(
    region='World',
    variable=df_for_equity.variable,
    subregions=df_for_equity.region,
    append=True
)

In [6]:
df_for_equity.require_variable(
    'Population',
    exclude_on_fail=True
)
df_for_equity.filter(
    exclude=False,
    inplace=True
)

pyam.core - INFO: 19 scenarios do not include required variable `Population`, marked as `exclude: True` in `meta`


Step 4: Now, calculate the fair shares from 2005.

In [7]:
results = []
for (mod, scen) in df_for_equity.meta.index:
    df_to_crunch = df_for_equity.filter(
        model=mod,
        scenario=scen
    )
    result = tecpc.calculate_carbon_debt(
        in_df=df_to_crunch,
        emission_var='AR6 Reanalysis|OSCARv3.2|Emissions|CO2|Direct Only',
        start_year=2010,
        end_year=2100
    )
    results.append(result)

In [8]:
results_pyam = pyam.concat(results)

Step 5: Pull out the year of global net zero CO2 from the original dataframe.

In [9]:
netzero_column = 'Year of netzero CO2 emissions (Harm-Infilled) table'

In [10]:
netzero_values = df_for_equity.meta.loc[:, netzero_column]

In [11]:
netzero_values = netzero_values.fillna(2100)

Step 6: First, add the cumulative debt between 2010 and 2019.

In [12]:
results_pyam.variable

['AR6 Reanalysis|OSCARv3.2|Emissions|CO2|Direct Only',
 'Emissions|Debt',
 'Emissions|Fair',
 'Population',
 'Population|Share']

In [13]:
for reg in results_pyam.region:
    if reg=='World':
        continue
    else:
        ts = (
            results_pyam
            .filter(
                variable='Emissions|Debt',
                region=reg
            )
            .timeseries()
        )
        results_pyam.set_meta(
            ts.apply(
                lambda x: pyam.cumulative(
                    x,
                    first_year=2010,
                    last_year=2019
                ),
                axis=1
            ),
            name=f'cumulative_debt_{reg}'
        )

Step 7: Now we need to add in the cumulative fair share on a per capita basis between 2020 and net zero (global).

In [14]:
for reg in results_pyam.region:
    if reg == 'World':
        continue
    else:
        ts = (
            results_pyam
            .filter(
                variable='Emissions|Fair',
                region=reg
            )
            .timeseries()
        )
        results_pyam.set_meta(
            ts.apply(
                lambda x: pyam.cumulative(
                    x,
                    first_year=2020,
                    last_year=results_pyam.meta.loc[x.name[0:2], netzero_column]
                ),
                axis=1
            ),
            name=f'cumulative_fair_{reg}'
        )

In [15]:
for reg in results_pyam.region:
    if reg=='World':
        continue
    else:
        results_pyam.meta.loc[:, f'benchmark_{reg}'] = (
            results_pyam.meta.loc[:, f'cumulative_fair_{reg}']
            -
            results_pyam.meta.loc[:, f'cumulative_debt_{reg}']
        )

In [16]:
cols_to_summarise = [
    x for x in results_pyam.meta.columns if 'benchmark' in x
]

Step 8: Pull out resulting fair shares and write this out.

In [17]:
fair_shares = (
    results_pyam
    .meta
    .groupby('Category')[cols_to_summarise]
    .describe()
    .T
    .round(0)
)

In [18]:
fair_shares.to_excel(
    Path(
        '../data/102_fair_share_benchmarks.xlsx'
    )
)